# Create lithogenic and biogenic particle fields for Pb scavenging

BLING ends in April, 2019

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from mpl_toolkits.basemap import Basemap
import netCDF4 as nc
import xarray as xr
import cmocean
from scipy.spatial import Delaunay
from scipy import ndimage as nd
from scipy.interpolate import LinearNDInterpolator

%matplotlib inline

#### Parameters / settings:

In [2]:
land_color = '#a9a7a2'

# domain dimensions:
jmin, jmax = 159, 799
imin, imax = 1139, 2179
isize = imax - imin
jsize = jmax - jmin

#### Load files

In [3]:
mesh         = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mask_Pb-20220317.nc')
mesh_lon     = mesh['nav_lon'].values
mesh_lat     = mesh['nav_lat'].values
mesh_bathy   = mesh['tmask'][0,:,:,:].values
bathy_masked = np.ma.masked_where((mesh_bathy> 0.1), mesh_bathy)
ocean_masked = np.ma.masked_where((mesh_bathy<0.1), mesh_bathy)
depths       = mesh['nav_lev'].values

In [4]:
# BLING:
folder_EPM101 = '/data/brogalla/ANHA4/BLING-EPM101/'
dset          = xr.open_dataset(f'/ocean/brogalla/GEOTRACES/BLING-EPM101/ANHA4-EPM101_y2002m08d13_gridB.nc')
ANHA4_lons    = dset['nav_lon'].values
ANHA4_lats    = dset['nav_lat'].values
ANHA4_depths  = dset['deptht'].values

In [5]:
# Mn for surface particle field:
dset_ANHA12 = xr.open_dataset('/data/brogalla/run_storage/Mn-extended-domain-202210/oMn_y2002m01.nc')
ANHA12_lons = mesh_lon[imin:imax,jmin:jmax]
ANHA12_lats = mesh_lat[imin:imax,jmin:jmax]

In [6]:
# interpolate from ANHA4 to ANHA12 grid:
tri = Delaunay(np.array([ANHA4_lons.flatten(), ANHA4_lats.flatten()]).transpose())  # Compute the triangulation

#### Functions:

In [7]:
def fill(data, invalid=None):
    """
    Replace the value of invalid 'data' cells (indicated by 'invalid') 
    by the value of the nearest valid data cell

    Input:
        data:    numpy array of any dimension
        invalid: a binary array of same shape as 'data'. True cells set where data
                 value should be replaced.
                 If None (default), use: invalid  = np.isnan(data)

    Output: 
        Return a filled array. 
    """

    if invalid is None: invalid = np.isnan(data)

    ind = nd.distance_transform_edt(invalid, return_distances=False, return_indices=True)
    return data[tuple(ind)]

In [8]:
def save_file(folder, litho_array, biomass_array, fpop_array, year, month):   
    
    file_write = xr.Dataset(
        {'lithogenic' : (("deptht","y","x"), litho_array),
         'biomass'    : (("deptht","y","x"), biomass_array),
         'fpop'       : (("deptht","y","x"), fpop_array)}, 
        coords = {
            "time_counter": np.zeros(1),
            "deptht": depths,
            "y": np.zeros(mesh_lat.shape[0]),
            "x": np.zeros(mesh_lon.shape[1])},
    )
    file_write.to_netcdf(f'{folder}Pb_scavenging_y{year}m{month:02}.nc', \
                         unlimited_dims='time_counter')

    return

In [9]:
def load_bio(year, month):
    # BLING:
    file_EPM101  = f'/data/brogalla/ANHA4/BLING-EPM101/ANHA4-EPM101_y{year}m{month:02}.nc'
    dset_bio     = xr.open_dataset(f'{file_EPM101}')
    fpop_ANHA4   = dset_bio['fpop'][:,:,:].values      # sinking particulate organic matter in phosphate units (mol P/m3)
    biop_ANHA4   = dset_bio['biomass_p'][:,:,:].values # biomass concentration in phosphate units (mol P/m3)
     
    # Convert sinking particulate organic matter in phosphate units by dividing by the sinking rate:
    # Sinking rate info comes from pg. 43-44 Laura Castro de la Guardia's thesis
    wsink              = np.ones(ANHA4_depths.shape)*16/(3600*24) # convert from m/day to m/s
    wsink[depths > 80] = (0.05*(ANHA4_depths[ANHA4_depths > 80]-80)+16)/(3600*24) 
    fpop_sink          = np.array([fpop_ANHA4[d,:,:] / wsink[d] for d in range(0,len(ANHA4_depths))])
    
    # Fill NaN and negative values with zero:
    fpop_sink[fpop_sink < 0]         = 0
    biop_ANHA4[biop_ANHA4 < 0]       = 0
    fpop_sink[np.isnan(fpop_sink)]   = np.nanmean(fpop_sink)
    biop_ANHA4[np.isnan(biop_ANHA4)] = np.nanmean(biop_ANHA4)
    
    # interpolate from ANHA4 to ANHA12 grid:
    ANHA12_biomass = np.empty((50,isize,jsize))
    ANHA12_fpop    = np.empty((50,isize,jsize))
    for depth in range(0,50):
        interpolator1 = LinearNDInterpolator(tri, biop_ANHA4[depth,:,:].flatten())  
        interpolator2 = LinearNDInterpolator(tri, fpop_sink[depth,:,:].flatten())    
    
        ANHA12_biomass[depth,:,:] = interpolator1(np.array([ANHA12_lons.flatten(), ANHA12_lats.flatten()]).transpose()).reshape(ANHA12_lons.shape)
        ANHA12_fpop[depth,:,:]    = interpolator2(np.array([ANHA12_lons.flatten(), ANHA12_lats.flatten()]).transpose()).reshape(ANHA12_lons.shape)

    return ANHA12_biomass, ANHA12_fpop

def load_litho(year, month):
    # Mn for surface particle field:
    folder_Mn   = f'/data/brogalla/run_storage/Mn-extended-domain-202210/'
    dset_ANHA12 = xr.open_dataset(f'{folder_Mn}oMn_y{year}m{month:02}.nc')
    ANHA12_Mn   = dset_ANHA12['oxidismn'][:,:,:].values 
    
    return ANHA12_Mn

In [10]:
def create_particle_fields(year, month, save=False):

    # initialize arrays with some small value, then fill with normalized particle fields:
    lithogenic_particles = np.zeros(mesh_bathy.shape)
    biomass_particles    = np.zeros(mesh_bathy.shape)
    fpop_particles       = np.zeros(mesh_bathy.shape)
    
    # load files:
    biomass_particles[:,imin:imax,jmin:jmax], fpop_particles[:,imin:imax,jmin:jmax] = load_bio(year, month)
    lithogenic_particles[:,imin:imax,jmin:jmax]                                     = load_litho(year, month)

    # Fill any weird values 
    biomass_particles_filled = fill(biomass_particles, invalid=(biomass_particles < 0))
    fpop_particles_filled    = fill(fpop_particles   , invalid=(fpop_particles < 0))
    biomass_particles_filled[(mesh_bathy < 0.1)]         = 0.0 # fill land with zeros
    fpop_particles_filled[(mesh_bathy < 0.1)]            = 0.0 # fill land with zeros
    lithogenic_particles[(mesh_bathy < 0.1)]             = 0.0 # fill land with zeros
    lithogenic_particles[lithogenic_particles < 0]       = 0.0
    lithogenic_particles[np.isnan(lithogenic_particles)] = 0.0
    
    if np.any(lithogenic_particles < 0):
        print('negative lithogenic')
    if np.any(biomass_particles_filled < 0):
        print('negative biomass')
    if np.any(fpop_particles_filled < 0):
        print('negative fpop')
    
    if save:
        folder ='/ocean/brogalla/GEOTRACES/data/Pb-tuning-202210/new-bio-proxy/general-particle-fields/'
        save_file(f'{folder}', lithogenic_particles, biomass_particles_filled, fpop_particles_filled, year, month)
    
    return

#### Calculations

Interpolate particle fields and save to file (without normalizing, making nonlinear, or applying beta):

In [12]:
for year in range(2019, 2020):
    for month in range(1, 13):
        print(year, month)
        create_particle_fields(year, month, save=True)

2019 1
2019 2
2019 3
2019 4


FileNotFoundError: [Errno 2] No such file or directory: b'/data/brogalla/ANHA4/BLING-EPM101/ANHA4-EPM101_y2019m04.nc'